# Deployment to Snowpark Container Service Demo

### Snowflake-ML-Python Installation

- Please refer to our [landing page](https://docs.snowflake.com/en/developer-guide/snowpark-ml/index) to install `snowflake-ml-python`.

## Train a model with Snowpark ML API 

In [14]:
from typing import Tuple
from snowflake.ml.modeling import linear_model
from sklearn import datasets
import pandas as pd
import numpy as np

def prepare_logistic_model() -> Tuple[linear_model.LogisticRegression, pd.DataFrame]:
    iris = datasets.load_iris()
    df = pd.DataFrame(data=np.c_[iris["data"], iris["target"]], columns=iris["feature_names"] + ["target"])
    df.columns = [s.replace(" (CM)", "").replace(" ", "") for s in df.columns.str.upper()]

    input_cols = ["SEPALLENGTH", "SEPALWIDTH", "PETALLENGTH", "PETALWIDTH"]
    label_cols = "TARGET"
    output_cols = "PREDICTED_TARGET"

    estimator = linear_model.LogisticRegression(
        input_cols=input_cols, output_cols=output_cols, label_cols=label_cols, random_state=0, max_iter=1000
    ).fit(df)

    return estimator, df.drop(columns=label_cols).head(10)

## Start Snowpark Session

In [12]:
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session

session = Session.builder.configs(SnowflakeLoginOptions()).create()


In [ ]:
from snowflake.ml.registry import model_registry
from snowflake.ml._internal.utils import identifier

db = identifier._get_unescaped_name(session.get_current_database())
schema = identifier._get_unescaped_name(session.get_current_schema())

# will be a no-op if registry already exists
model_registry.create_model_registry(session=session, database_name=db, schema_name=schema) 
registry = model_registry.ModelRegistry(session=session, database_name=db, schema_name=schema)

## Register SnowML Model

In [11]:
logistic_model, test_features = prepare_logistic_model()
model_name = "snowpark_ml_logistic"
model_version = "v1"

model_ref = registry.log_model(
    model_name=model_name,
    model_version=model_version,
    model=logistic_model,
    sample_input_data=test_features,
)

'0aa236602be711ee89915ac3f3b698e1'

## Model Deployment to Snowpark Container Service

In [6]:
from snowflake.ml.model import deploy_platforms
from snowflake import snowpark

compute_pool = "MY_COMPUTE_POOL" # Pre-created compute pool
deployment_name = "LOGISTIC_FUNC" # Name of the resulting UDF

model_ref.deploy(
    deployment_name=deployment_name, 
    platform=deploy_platforms.TargetPlatform.SNOWPARK_CONTAINER_SERVICES,
    target_method="predict",
    options={
        "compute_pool": compute_pool,
        #num_gpus: 1 # Specify the number of GPUs for GPU inferenc
    }
)

## Batch Prediction on Snowpark Container Service

In [7]:
model_ref.predict(deployment_name, test_features)

,SEPALLENGTH,SEPALWIDTH,PETALLENGTH,PETALWIDTH,PREDICTED_TARGET
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
5,5.4,3.9,1.7,0.4,0.0
6,4.6,3.4,1.4,0.3,0.0
7,5.0,3.4,1.5,0.2,0.0
8,4.4,2.9,1.4,0.2,0.0
9,4.9,3.1,1.5,0.1,0.0


In [ ]:
model_ref.delete_deployment(deployment_name=deployment_name)

In [ ]:
model_ref.delete_model()